## Merit-order Power Dispatch Optimization

Electricity systems must balance **generation and demand** at every moment. \
Each power plant has a different **marginal cost** - or the cost of producing _one more_ kWh. \
When demand rises, the system operator calls plants in ascending order of marginal cost: the so-called, **merit-order**

The market price $(p^*)$ equals the cost of the **last unit dispatched**, and every generator running at that time receives this price. \
Low-cost plants earn extra revenue ('infra-marginal rent'), while high-cost plants risk the _missing-money problem_ when prices are too low to cover fixed costs.

In optimization language, this can be represented as a **linear program**:

$\min_{g_i} \sum_i c_i g_i$

subject to
$\sum_i g_i = D, \qquad 0 \le g_i \le \text{Cap}_i$

where:
- g_i = generation from plant i (MWh)
- c_i = marginal cost (€/MWh)
- D = total demand (MWh)

The result gives which plants operate and at what level

In [1]:
import pulp as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

### Baseline merit-order dispatch LP (no network)

In [3]:
# Sets
# Plants
plants = [
    "Wind_1",
    "Wind_2",
    "Nuclear_1",
    "Biomass_1",
    "Coal_1",
    "Gas_1",
    "Gas_2"
]

# Parameters
marginal_cost = {
    "Wind_1": 0,
    "Wind_2": 0,
    "Nuclear_1": 10,
    "Biomass_1": 40,
    "Coal_1": 60,
    "Gas_1": 80,
    "Gas_2": 100
}

# Available capacity [MW] each plant can generate this hour
capacity = {
    "Wind_1": 100,
    "Wind_2": 100,
    "Nuclear_1": 200,
    "Biomass_1": 150,
    "Coal_1": 250,
    "Gas_1": 150,
    "Gas_2": 150
}

# System demand [MW] that must be met
demand = 750  # as agreed

In [4]:
# Model
model = pl.LpProblem('least_cost_dispatch', pl.LpMinimize)

# Decision variables: how much will each plant generates ? in [MW]
gen = {
    p: pl.LpVariable(f'Gen_{p}', lowBound=0,
                     upBound=capacity[p], cat='Continuous')
    for p in plants
}

# Objective: minimize total operating cost
model += pl.lpSum(marginal_cost[p] * gen[p] for p in plants)

# Demand balance: total generation must meet demand exactly
model += pl.lpSum(gen[p] for p in plants) == demand, 'DemandBalance'

# Solve
model.solve(pl.PULP_CBC_CMD(msg=True))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0f/969vx_sj1zq24nf1010z0vhw0000gn/T/a9592125090f4015970b028ea1ef4709-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0f/969vx_sj1zq24nf1010z0vhw0000gn/T/a9592125090f4015970b028ea1ef4709-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 19 RHS
At line 21 BOUNDS
At line 29 ENDATA
Problem MODEL has 1 rows, 7 columns and 7 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (0) rows, 5 (-2) columns and 5 (-2) elements
0  Obj 0 Primal inf 550 (1)
1  Obj 20000
Optimal - objective value 20000
After Postsolve, objective 20000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 20000 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal t

1

In [5]:
# Report results
print(f"Status: {pl.LpStatus[model.status]}")
total_cost = pl.value(model.objective)
print(f"Total Operating Cost: €{total_cost:.2f}")

print("\nDispatch by plant [MW]:")
for p in plants:
    print(
        f"  {p:<10} : {gen[p].value():6.2f} / {capacity[p]} MW (cost {marginal_cost[p]} €/MWh)")

# Infer clearing price (merit order logic)
# The market-clearing price in a uniform-price market is the marginal cost
# of the last plant that is (a) dispatched >0 and (b) not fully unlimited cheap.
# Practically: it's the highest marginal cost among plants with gen>0.
active_plants = [p for p in plants if gen[p].value() > 1e-6]
clearing_price = max(marginal_cost[p] for p in active_plants)
print(f"\nImplied market clearing price p*: {clearing_price} €/MWh")

Status: Optimal
Total Operating Cost: €20000.00

Dispatch by plant [MW]:
  Wind_1     : 100.00 / 100 MW (cost 0 €/MWh)
  Wind_2     : 100.00 / 100 MW (cost 0 €/MWh)
  Nuclear_1  : 200.00 / 200 MW (cost 10 €/MWh)
  Biomass_1  : 150.00 / 150 MW (cost 40 €/MWh)
  Coal_1     : 200.00 / 250 MW (cost 60 €/MWh)
  Gas_1      :   0.00 / 150 MW (cost 80 €/MWh)
  Gas_2      :   0.00 / 150 MW (cost 100 €/MWh)

Implied market clearing price p*: 60 €/MWh


### Networked power system (with grid/congestion)

We're stepping up our game, by introducing space/location/geography.

Because, the time we say 'Java vs Sumatera vs Nusa tenggara', we're no longer solving one big pooled market. \
We are solving a *network-constrained* power flow, with a possibility of transmission bottleneck. \
That's what infra investors, TSOs, and consultants actually care about.

In [8]:
def solve_power_system(capacity, demand_region):
    regions = ["North", "Central", "South"]

    plants = [
        "Wind_1",
        "Wind_2",
        "Nuclear_1",
        "Biomass_1",
        "Coal_1",
        "Gas_1",
        "Gas_2"
    ]

    plant_region = {
        "Wind_1": "North",
        "Nuclear_1": "North",
        "Wind_2": "Central",
        "Biomass_1": "Central",
        "Coal_1": "South",
        "Gas_1": "South",
        "Gas_2": "South"
    }

    marginal_cost = {
        "Wind_1": 0,
        "Wind_2": 0,
        "Nuclear_1": 10,
        "Biomass_1": 40,
        "Coal_1": 60,
        "Gas_1": 80,
        "Gas_2": 100
    }

    # Directed transmission arcs
    arcs = [
        ("North", "Central"),
        ("Central", "North"),
        ("Central", "South"),
        ("South", "Central")
    ]

    line_capacity = {
        ("North", "Central"): 200,
        ("Central", "North"): 200,
        ("Central", "South"): 150,
        ("South", "Central"): 150
    }

    # Build the model
    model = pl.LpProblem("network_dispatch", pl.LpMinimize)

    # Generation decision variables
    gen = {
        p: pl.LpVariable(
            f"Gen_{p}",
            lowBound=0,
            upBound=capacity[p],
            cat="Continuous"
        )
        for p in plants
    }

    # Flow decision variables between regions
    flow = {
        (src, dst): pl.LpVariable(
            f"Flow_{src}_to_{dst}",
            lowBound=0,
            upBound=line_capacity[(src, dst)],
            cat="Continuous"
        )
        for (src, dst) in arcs
    }

    # Objective: minimize total generation cost
    model += pl.lpSum(marginal_cost[p] * gen[p] for p in plants)

    # Regional nodal balance:
    # sum of generation in region + sum of inflows from other regions - sum of outflows to other regions == demand in that region
    for region in regions:
        gen_in_region = pl.lpSum(
            gen[p] for p in plants if plant_region[p] == region
        )

        inflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if dst == region
        )

        outflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if src == region
        )

        model += (
            gen_in_region + inflow_region - outflow_region
            == demand_region[region]
        ), f"Balance_{region}"

    # Solve
    model.solve(pl.PULP_CBC_CMD(msg=False))

    # Report
    print(f"Status: {pl.LpStatus[model.status]}")
    system_cost = pl.value(model.objective)
    print(f"Total System Cost: €{system_cost:.2f}\n")

    print("Dispatch by plant [MW]:")
    for p in plants:
        print(
            f"  {p:<10} : {gen[p].value():6.2f} / {capacity[p]} MW "
            f"(cost {marginal_cost[p]} €/MWh, region {plant_region[p]})"
        )

    print("\nFlows between regions [MW]:")
    for (src, dst) in arcs:
        val = flow[(src, dst)].value()
        if val > 1e-6:
            print(
                f"  {src} → {dst}: {val:.2f} MW "
                f"(limit {line_capacity[(src, dst)]} MW)"
            )

    print("\nRegional supply/demand check:")
    for region in regions:
        gen_val = sum(
            gen[p].value() for p in plants if plant_region[p] == region
        )
        inflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if dst == region
        )
        outflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if src == region
        )
        lhs = gen_val + inflow_val - outflow_val
        print(
            f"  {region:<7} demand {demand_region[region]:6.2f}  |  "
            f"LHS {lhs:6.2f}  "
            f"(gen {gen_val:.2f} + inflow {inflow_val:.2f} - outflow {outflow_val:.2f})"
        )

    return model, gen, flow

The model now decides:
1. how much each plant generates,
2. how much power flows between regions,
3.	and must satisfy demand in each region, not just globally.

In [9]:
cap_base = {
    "Wind_1": 100,
    "Wind_2": 100,
    "Nuclear_1": 200,
    "Biomass_1": 150,
    "Coal_1": 250,
    "Gas_1": 150,
    "Gas_2": 150
}

demand_base = {
    "North":   300,
    "Central": 250,
    "South":   200
}

solve_power_system(cap_base, demand_base)

Status: Optimal
Total System Cost: €20000.00

Dispatch by plant [MW]:
  Wind_1     : 100.00 / 100 MW (cost 0 €/MWh, region North)
  Wind_2     : 100.00 / 100 MW (cost 0 €/MWh, region Central)
  Nuclear_1  : 200.00 / 200 MW (cost 10 €/MWh, region North)
  Biomass_1  : 150.00 / 150 MW (cost 40 €/MWh, region Central)
  Coal_1     : 200.00 / 250 MW (cost 60 €/MWh, region South)
  Gas_1      :   0.00 / 150 MW (cost 80 €/MWh, region South)
  Gas_2      :   0.00 / 150 MW (cost 100 €/MWh, region South)

Flows between regions [MW]:

Regional supply/demand check:
  North   demand 300.00  |  LHS 300.00  (gen 300.00 + inflow 0.00 - outflow 0.00)
  Central demand 250.00  |  LHS 250.00  (gen 250.00 + inflow 0.00 - outflow 0.00)
  South   demand 200.00  |  LHS 200.00  (gen 200.00 + inflow 0.00 - outflow 0.00)


(network_dispatch:
 MINIMIZE
 40*Gen_Biomass_1 + 60*Gen_Coal_1 + 80*Gen_Gas_1 + 100*Gen_Gas_2 + 10*Gen_Nuclear_1 + 0.0
 SUBJECT TO
 Balance_North: Flow_Central_to_North - Flow_North_to_Central + Gen_Nuclear_1
  + Gen_Wind_1 = 300
 
 Balance_Central: - Flow_Central_to_North - Flow_Central_to_South
  + Flow_North_to_Central + Flow_South_to_Central + Gen_Biomass_1 + Gen_Wind_2
  = 250
 
 Balance_South: Flow_Central_to_South - Flow_South_to_Central + Gen_Coal_1
  + Gen_Gas_1 + Gen_Gas_2 = 200
 
 VARIABLES
 Flow_Central_to_North <= 200 Continuous
 Flow_Central_to_South <= 150 Continuous
 Flow_North_to_Central <= 200 Continuous
 Flow_South_to_Central <= 150 Continuous
 Gen_Biomass_1 <= 150 Continuous
 Gen_Coal_1 <= 250 Continuous
 Gen_Gas_1 <= 150 Continuous
 Gen_Gas_2 <= 150 Continuous
 Gen_Nuclear_1 <= 200 Continuous
 Gen_Wind_1 <= 100 Continuous
 Gen_Wind_2 <= 100 Continuous,
 {'Wind_1': Gen_Wind_1,
  'Wind_2': Gen_Wind_2,
  'Nuclear_1': Gen_Nuclear_1,
  'Biomass_1': Gen_Biomass_1,
  'Coa

### Note:

If you look closely, no intra-region transmission is actually happening. \
North, for example, can already fulfill his need. Why should it imports? \
\
Each zone has its own generation fleet sized to cover its own peak. Interconnection cables exist, but under normal conditions they’re not doing anything. TSOs would call this a low-stress hour.
\
\
It’s actually a boring hour

### Let's tweak again!
We gonna change south's demand to 300, and add some spare capacity on North.

In [10]:
cap_upgrade = {
    "Wind_1": 100,
    "Wind_2": 100,
    "Nuclear_1": 300,  # upgraded
    "Biomass_1": 150,
    "Coal_1": 250,
    "Gas_1": 150,
    "Gas_2": 150
}

demand_stress = {
    "North":   300,
    "Central": 250,
    "South":   300
}

solve_power_system(cap_upgrade, demand_stress)

Status: Optimal
Total System Cost: €21000.00

Dispatch by plant [MW]:
  Wind_1     : 100.00 / 100 MW (cost 0 €/MWh, region North)
  Wind_2     : 100.00 / 100 MW (cost 0 €/MWh, region Central)
  Nuclear_1  : 300.00 / 300 MW (cost 10 €/MWh, region North)
  Biomass_1  : 150.00 / 150 MW (cost 40 €/MWh, region Central)
  Coal_1     : 200.00 / 250 MW (cost 60 €/MWh, region South)
  Gas_1      :   0.00 / 150 MW (cost 80 €/MWh, region South)
  Gas_2      :   0.00 / 150 MW (cost 100 €/MWh, region South)

Flows between regions [MW]:
  North → Central: 100.00 MW (limit 200 MW)
  Central → South: 100.00 MW (limit 150 MW)

Regional supply/demand check:
  North   demand 300.00  |  LHS 300.00  (gen 400.00 + inflow 0.00 - outflow 100.00)
  Central demand 250.00  |  LHS 250.00  (gen 250.00 + inflow 100.00 - outflow 100.00)
  South   demand 300.00  |  LHS 300.00  (gen 200.00 + inflow 100.00 - outflow 0.00)


(network_dispatch:
 MINIMIZE
 40*Gen_Biomass_1 + 60*Gen_Coal_1 + 80*Gen_Gas_1 + 100*Gen_Gas_2 + 10*Gen_Nuclear_1 + 0.0
 SUBJECT TO
 Balance_North: Flow_Central_to_North - Flow_North_to_Central + Gen_Nuclear_1
  + Gen_Wind_1 = 300
 
 Balance_Central: - Flow_Central_to_North - Flow_Central_to_South
  + Flow_North_to_Central + Flow_South_to_Central + Gen_Biomass_1 + Gen_Wind_2
  = 250
 
 Balance_South: Flow_Central_to_South - Flow_South_to_Central + Gen_Coal_1
  + Gen_Gas_1 + Gen_Gas_2 = 300
 
 VARIABLES
 Flow_Central_to_North <= 200 Continuous
 Flow_Central_to_South <= 150 Continuous
 Flow_North_to_Central <= 200 Continuous
 Flow_South_to_Central <= 150 Continuous
 Gen_Biomass_1 <= 150 Continuous
 Gen_Coal_1 <= 250 Continuous
 Gen_Gas_1 <= 150 Continuous
 Gen_Gas_2 <= 150 Continuous
 Gen_Nuclear_1 <= 300 Continuous
 Gen_Wind_1 <= 100 Continuous
 Gen_Wind_2 <= 100 Continuous,
 {'Wind_1': Gen_Wind_1,
  'Wind_2': Gen_Wind_2,
  'Nuclear_1': Gen_Nuclear_1,
  'Biomass_1': Gen_Biomass_1,
  'Coa

Now, we can see that even though North only need 300 MW, they produce 400 MW, and export the 100 MW! \
That's because the bid price (from the north) is so low, that the model utilize it to maximum capacity, rather than have to ramp up more in the central.

But! Why does central also utilize all of its biomass, even though their current demand can be fulfilled with the help of north? \
Well, because they export it also to... south. That way, we manage not to trigger the gas ramp-up.